In [ ]:
import numpy, scipy, sklearn
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("sklearn:", sklearn.__version__)

In [ ]:
# %load simple_flask_app.py
#!/usr/bin/env python3
"""
Simplified Flask app for Social Media Sentiment Analysis Platform
This version works with basic dependencies and demonstrates core functionality.
"""

from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
import json
import time
from datetime import datetime
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
try:
    import tensorflow as tf
    TF_AVAILABLE = Truej
except ModuleNotFoundError:
    tf = None
    TF_AVAILABLE = False

app = Flask(__name__)
CORS(app)

# Simple Text Preprocessor (same as in simple_demo.py)
class SimpleTextPreprocessor:
    def __init__(self):
        try:
            nltk.data.find('tokenizers/punkt_tab')
            nltk.data.find('corpora/stopwords')
            nltk.data.find('corpora/wordnet')
        except LookupError:
            nltk.download('punkt_tab')
            nltk.download('stopwords')
            nltk.download('wordnet')

        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))

    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'@\w+|#\w+', '', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def tokenize_and_lemmatize(self, text):
        tokens = word_tokenize(text)
        lemmatized_tokens = [
            self.lemmatizer.lemmatize(token)
            for token in tokens
            if token not in self.stop_words and len(token) > 2
        ]
        return lemmatized_tokens

    def preprocess(self, text):
        cleaned_text = self.clean_text(text)
        tokens = self.tokenize_and_lemmatize(cleaned_text)
        return ' '.join(tokens)

# Simple Sentiment Analyzer
class SimpleSentimentAnalyzer:
    def __init__(self):
        self.models = {}
        self.vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
        self.preprocessor = SimpleTextPreprocessor()
        self.is_trained = False
        self.sentiment_labels = ['Negative', 'Neutral', 'Positive']
        self.neural_enabled = TF_AVAILABLE

        # Initialize with sample data for demo
        self._train_with_sample_data()

    def _build_neural_network(self, input_dim, num_classes=3):
        """Create a lightweight dense neural network for demo purposes."""
        if not self.neural_enabled or tf is None:
            raise RuntimeError("TensorFlow is required for the neural network demo.")
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_dim,)),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    def _train_neural_network(self, X, y):
        """Train the neural network on dense TF-IDF features."""
        if not self.neural_enabled or tf is None:
            return None
        X_dense = X.toarray().astype('float32')
        y_array = np.array(y, dtype=np.int32)
        model = self._build_neural_network(X_dense.shape[1], len(self.sentiment_labels))
        # Small dataset -> keep epochs low to avoid overfitting while still demonstrating NN usage
        model.fit(X_dense, y_array, epochs=25, batch_size=16, verbose=0)
        return model

    def _predict_neural_network(self, model, X):
        """Predict sentiment probabilities with the neural network."""
        X_dense = X.toarray().astype('float32')
        probabilities = model.predict(X_dense, verbose=0)[0]
        prediction = int(np.argmax(probabilities))
        return prediction, probabilities

    def _train_with_sample_data(self):
        """Train models with sample data for demonstration"""
        sample_tweets = [
            "I absolutely love this new product! It's amazing!",
            "This is the worst service I've ever experienced. Terrible!",
            "The weather is okay today, nothing special.",
            "Best purchase I've made this year! Highly recommend!",
            "Not impressed with the quality. Could be better.",
            "Feeling great today! Life is beautiful!",
            "The movie was boring and too long. Waste of time.",
            "Pretty good overall experience. Satisfied with the results.",
            "Absolutely hate this new update. Ruined everything!",
            "Amazing customer service! They went above and beyond!",
            "This is confusing and hard to use.",
            "Love the new features! Great job!",
            "It's fine, works as expected.",
            "Incredible performance! Exceeded expectations!",
            "Terrible experience. Very disappointed.",
            "Outstanding quality and service!",
            "Could be much better, disappointed.",
            "Average product, nothing special.",
            "Fantastic experience, will definitely return!",
            "Poor quality, would not recommend."
        ]

        expected_labels = [2, 0, 1, 2, 0, 2, 0, 2, 0, 2, 0, 2, 1, 2, 0, 2, 0, 1, 2, 0]

        # Preprocess and vectorize
        preprocessed_texts = [self.preprocessor.preprocess(text) for text in sample_tweets]
        X = self.vectorizer.fit_transform(preprocessed_texts)
        y = np.array(expected_labels)

        # Train models
        self.models['naive_bayes'] = MultinomialNB().fit(X, y)
        self.models['svm'] = SVC(probability=True, random_state=42).fit(X, y)
        self.models['logistic_regression'] = LogisticRegression(random_state=42, max_iter=1000).fit(X, y)
        self.models['random_forest'] = RandomForestClassifier(
            n_estimators=200,
            max_depth=None,
            random_state=42
        ).fit(X, y)
        if self.neural_enabled:
            self.models['neural_network'] = self._train_neural_network(X, y)
        else:
            print("⚠️ TensorFlow not installed; skipping neural network demo. Run 'pip install tensorflow' to enable it.")

        self.is_trained = True

    def analyze_text(self, text, model_name='svm'):
        """Analyze sentiment of a single text"""
        if not self.is_trained:
            return {'error': 'Models are still initializing. Please try again in a moment.'}

        if model_name == 'neural_network' and not self.neural_enabled:
            return {'error': "Neural network demo disabled because TensorFlow isn't installed. Run 'pip install tensorflow' to enable it."}

        if model_name not in self.models:
            return {'error': f"Model '{model_name}' not available"}

        # Preprocess text
        preprocessed_text = self.preprocessor.preprocess(text)

        # Transform to features
        X = self.vectorizer.transform([preprocessed_text])

        # Make prediction
        model = self.models[model_name]
        if model_name == 'neural_network':
            prediction, probabilities = self._predict_neural_network(model, X)
        else:
            if hasattr(model, 'predict_proba'):
                probabilities = model.predict_proba(X)[0]
                prediction = int(np.argmax(probabilities))
            else:
                prediction = int(model.predict(X)[0])
                probabilities = None

        confidence = float(np.max(probabilities) * 100) if probabilities is not None else None

        return {
            'text': text,
            'sentiment': self.sentiment_labels[prediction],
            'sentiment_score': int(prediction),
            'confidence': round(confidence, 1) if confidence is not None else None,
            'probabilities': self._format_probabilities(probabilities),
            'model_used': model_name
        }

    def compare_models(self, text):
        """Compare results across all models"""
        results = {}
        for model_name in self.models.keys():
            results[model_name] = self.analyze_text(text, model_name)
        return results

    def _format_probabilities(self, probabilities):
        """Convert probability vectors into a friendly dict for JSON responses."""
        if probabilities is None:
            return None
        probs = [round(float(p) * 100, 1) for p in probabilities]
        while len(probs) < 3:
            probs.append(0.0)
        return {
            'negative': probs[0],
            'neutral': probs[1],
            'positive': probs[2]
        }

# Initialize the sentiment analyzer
sentiment_analyzer = SimpleSentimentAnalyzer()

# Simple HTML template for the web interface
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Social Media Sentiment Analysis</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; background-color: #f5f5f5; }
        .container { max-width: 800px; margin: 0 auto; background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
        h1 { color: #333; text-align: center; margin-bottom: 30px; }
        .input-section { margin-bottom: 30px; }
        textarea { width: 100%; height: 100px; padding: 10px; border: 1px solid #ddd; border-radius: 5px; font-size: 14px; }
        button { background: #007bff; color: white; padding: 10px 20px; border: none; border-radius: 5px; cursor: pointer; font-size: 16px; margin: 10px 5px 0 0; }
        button:hover { background: #0056b3; }
        .results { margin-top: 20px; }
        .result-box { background: #f8f9fa; padding: 15px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #007bff; }
        .positive { border-left-color: #28a745; }
        .negative { border-left-color: #dc3545; }
        .neutral { border-left-color: #ffc107; }
        .confidence { font-weight: bold; }
        .probabilities { font-size: 12px; color: #666; margin-top: 5px; }
        .demo-buttons { text-align: center; margin: 20px 0; }
        .demo-text { background: #e9ecef; padding: 10px; border-radius: 5px; margin: 5px 0; cursor: pointer; }
        .demo-text:hover { background: #dee2e6; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎭 Social Media Sentiment Analysis</h1>

        <div class="demo-buttons">
            <h3>Try some examples:</h3>
            <div class="demo-text" onclick="setDemoText('I absolutely love this new product! Amazing quality and service!')">
                😊 "I absolutely love this new product! Amazing quality and service!"
            </div>
            <div class="demo-text" onclick="setDemoText('This is terrible service. Very disappointed and frustrated.')">
                😞 "This is terrible service. Very disappointed and frustrated."
            </div>
            <div class="demo-text" onclick="setDemoText('The product is okay, nothing special but does what it says.')">
                😐 "The product is okay, nothing special but does what it says."
            </div>
        </div>

        <div class="input-section">
            <textarea id="textInput" placeholder="Enter text to analyze sentiment... (e.g., tweets, reviews, comments)"></textarea>
            <br>
            <button onclick="analyzeSingle()">Analyze with SVM</button>
            <button onclick="compareModels()">Compare All Models</button>
            <button onclick="clearResults()">Clear Results</button>
        </div>

        <div id="results" class="results"></div>
    </div>

    <script>
        function setDemoText(text) {
            document.getElementById('textInput').value = text;
        }

        function analyzeSingle() {
            const text = document.getElementById('textInput').value;
            if (!text.trim()) {
                alert('Please enter some text to analyze');
                return;
            }

            fetch('/api/analyze', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({text: text, model: 'svm'})
            })
            .then(response => response.json())
            .then(data => displaySingleResult(data))
            .catch(error => console.error('Error:', error));
        }

        function compareModels() {
            const text = document.getElementById('textInput').value;
            if (!text.trim()) {
                alert('Please enter some text to analyze');
                return;
            }

            fetch('/api/compare', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({text: text})
            })
            .then(response => response.json())
            .then(data => displayComparisonResults(data))
            .catch(error => console.error('Error:', error));
        }

        function displaySingleResult(data) {
            const resultsDiv = document.getElementById('results');
            const sentimentClass = data.sentiment.toLowerCase();

            resultsDiv.innerHTML = `
                <div class="result-box ${sentimentClass}">
                    <h3>Analysis Result</h3>
                    <p><strong>Text:</strong> "${data.text}"</p>
                    <p><strong>Sentiment:</strong> ${data.sentiment} <span class="confidence">(${data.confidence}% confidence)</span></p>
                    <p><strong>Model:</strong> ${data.model_used.replace('_', ' ').toUpperCase()}</p>
                    <div class="probabilities">
                        Probabilities: Negative ${data.probabilities.negative}% |
                        Neutral ${data.probabilities.neutral}% |
                        Positive ${data.probabilities.positive}%
                    </div>
                </div>
            `;
        }

        function displayComparisonResults(data) {
            const resultsDiv = document.getElementById('results');
            let html = '<h3>Model Comparison Results</h3>';

            for (const [modelName, result] of Object.entries(data)) {
                const sentimentClass = result.sentiment.toLowerCase();
                html += `
                    <div class="result-box ${sentimentClass}">
                        <h4>${modelName.replace('_', ' ').toUpperCase()}</h4>
                        <p><strong>Sentiment:</strong> ${result.sentiment} <span class="confidence">(${result.confidence}% confidence)</span></p>
                        <div class="probabilities">
                            Probabilities: Negative ${result.probabilities.negative}% |
                            Neutral ${result.probabilities.neutral}% |
                            Positive ${result.probabilities.positive}%
                        </div>
                    </div>
                `;
            }

            resultsDiv.innerHTML = html;
        }

        function clearResults() {
            document.getElementById('results').innerHTML = '';
            document.getElementById('textInput').value = '';
        }
    </script>
</body>
</html>
"""

# Routes
@app.route('/')
def index():
    """Main page"""
    return render_template_string(HTML_TEMPLATE)

@app.route('/api/health')
def health():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat(),
        'models_available': list(sentiment_analyzer.models.keys()),
        'models_trained': sentiment_analyzer.is_trained
    })

@app.route('/api/analyze', methods=['POST'])
def analyze_sentiment():
    """Analyze sentiment of text"""
    try:
        data = request.get_json()
        text = data.get('text', '')
        model = data.get('model', 'svm')

        if not text.strip():
            return jsonify({'error': 'Text is required'}), 400

        result = sentiment_analyzer.analyze_text(text, model)
        return jsonify(result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/compare', methods=['POST'])
def compare_models():
    """Compare sentiment analysis across all models"""
    try:
        data = request.get_json()
        text = data.get('text', '')

        if not text.strip():
            return jsonify({'error': 'Text is required'}), 400

        results = sentiment_analyzer.compare_models(text)
        return jsonify(results)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/models')
def get_models():
    """Get available models"""
    return jsonify({
        'models': list(sentiment_analyzer.models.keys()),
        'default_model': 'svm',
        'sentiment_labels': sentiment_analyzer.sentiment_labels
    })

if __name__ == '__main__':
    print("🚀 Starting Social Media Sentiment Analysis Web App...")
    print("📊 Models available:", list(sentiment_analyzer.models.keys()))
    print("🌐 Web interface will be available at: http://localhost:5001")
    print("🔍 API endpoints:")
    print("  - GET  /api/health")
    print("  - POST /api/analyze")
    print("  - POST /api/compare")
    print("  - GET  /api/models")
    print("\n✨ Ready to analyze sentiment! Open http://localhost:5001 in your browser")

    app.run(debug=True, host='0.0.0.0', port=5001)


In [ ]:
import sys, platform
py = sys.executable

# Keep build tools fresh
!{py} -m pip install -U pip setuptools wheel

# Choose the right TF for your Mac
if platform.system() == "Darwin" and platform.machine() == "arm64":
    # Apple Silicon: use macOS build + Metal acceleration
    !{py} -m pip install -U "tensorflow-macos>=2.16,<3" tensorflow-metal
else:
    # Intel Mac / other OS
    !{py} -m pip install -U "tensorflow>=2.16,<3"

In [ ]:
if __name__ == "__main__":
    # Keep your host/port as you like; the key is turning these OFF in Jupyter.
    app.run(host="127.0.0.1", port=5001, debug=False, use_reloader=False)